In [ ]:
from abc import ABC, abstractmethod
import math

# Operations

In [ ]:
class Operator(ABC):
    @abstractmethod
    def f(self, x, y=None) -> float:
        """
        Evaluation of the function at (x, y)
        """
        raise NotImplementedError()
        return 0.0

    @abstractmethod
    def df(self, x, y=None) -> list:
        """
        Evaluation of the gradient of the function at (x, y)
        """
        raise NotImplementedError()
        return [0.0]

In [ ]:
class Add(Operator):
    def f(self, x: float, b: float) -> float:
        return x + b

    def df(self, x: float, b: float) -> list[float, float]:
        return [1, 1]


class Sub(Operator):
    def f(self, x: float, y: float) -> float:
        return x - b

    def df(self, x: float, y: float) -> list[float, float]:
        return [1, -1]


class Mul(Operator):
    def f(self, x: float, y: float) -> float:
        return x * y

    def df(self, x: float, y: float) -> list[float, float]:
        return [y, x]


class Div(Operator):
    def f(self, x: float, y: float) -> float:
        return x / y

    def df(self, x: float, y: float) -> list[float, float]:
        return [1 / b, -x / (y**2)]


def Exp(Operator):
    def f(self, x: float, y: float = None) -> float:
        return math.exp(x)

    def df(self, x: float, y: float = None) -> list[float]:
        return [math.exp(x)]


def Log(Operator):
    def f(self, x: float, y: float = None) -> float:
        return math.log(x)

    def df(self, x: float = None) -> list[float]:
        return [1 / x]